<a href="https://colab.research.google.com/github/kordusmonika/dw_matrix_car/blob/master/day5_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import xgboost as xgb
from hyperopt import hp, fmin, tpe, STATUS_OK

from sklearn.metrics import mean_absolute_error

import eli5
from eli5.sklearn import PermutationImportance

In [4]:
cd /content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_2/dw_matrix_car/data

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_2/dw_matrix_car/data


In [5]:
df = pd.read_hdf('car.h5')
df.shape

(106494, 155)

In [0]:
SUFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[ feat + SUFIX_CAT ] = factorized_values

In [10]:
cat_feats = [x for x in df.columns if SUFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x) )
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]) )
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ', '') ) )

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [16]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}

feats = ['param_napęd__cat',
'param_stan__cat',
'param_faktura-vat__cat',
'param_rok-produkcji',
'param_moc',
'param_skrzynia-biegów__cat',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'param_pojemność-skokowa',
'seller_name__cat',
'param_wersja__cat',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'feature_system-start-stop__cat',
'param_kod-silnika__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_światła-led__cat', 
'feature_czujniki-parkowania-przednie__cat']

run_model(xgb.XGBRegressor(**xgb_params), feats )

[10:47:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:47:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:47:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9570.85281125158, 87.36178156613644)

#Hyperopt ~7429

In [19]:
def obj_func(params):

  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)

  return {'loss': np.abs(mean_mae), 'status':STATUS_OK }

xgb_reg_params = {
    'learning_rate': hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)),
    'max_depth':hp.choice('max_depth', np.arange(5,16,1, dtype=int)),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective': 'reg:squarederror',
    'n_estimators': 100,
    'seed': 0,
}

#run
best = fmin(
    obj_func,
    xgb_reg_params,
    algo=tpe.suggest,
    max_evals = 20
)

best

100%|██████████| 20/20 [15:40<00:00, 47.03s/it, best loss: 7429.429004725335]


{'colsample_bytree': 0.8, 'learning_rate': 1, 'max_depth': 8, 'subsample': 1.0}